In [1]:
import pandas as pd
import numpy as np
import spacy
import requests
import matplotlib.pyplot as plt
from requests.auth import HTTPBasicAuth

In [11]:
client_id = 'kjQMGYHYrtcO056jMFxJkQ'
secret_token = 'k6K7tDWwv0Ph_4091BCE63NO4Rm-pA'
auth = HTTPBasicAuth(client_id, secret_token)

In [106]:
def refresh_token(clien_id=client_id, client_secret=secret_token):
    auth = HTTPBasicAuth(client_id, secret_token)
    payload = {"grant_type": "password", "username": "anthony_tobi", "password": "oluwatobi2050"}
    headers = {"User-Agent": "Roberta-Basev0.0.1 by anthony_tobi"}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=auth, data=payload, headers=headers)
    headers["Authorization"] = f"bearer {response.json()['access_token']}"
    url = "https://oauth.reddit.com"
    return headers, url

In [111]:
headers, url = refresh_token(clien_id=client_id, client_secret=secret_token)

In [119]:
def get_reddit_data(subreddit="Investing",url=url, headers=headers):
    res = requests.get(f"{url}/r/{subreddit}/new", headers=headers, params={"limit": 100})
    data = pd.DataFrame({'name': [], 'title':[], 'selftext':[], 'category':[], 'upvote_ratio':[]})

    while True:
        try:
            for post in res.json()['data']['children']:
                # Convert the dictionary to a data frame
                dict_to_append = {'name': post['data']['name'], 'title':post['data']['title'], 'selftext': post['data']['selftext'], 
                                  'category': post['data']['category'],'upvote_ratio': post['data']['upvote_ratio']}
                # print(dict_to_append['name'])
                df_to_append = pd.DataFrame(dict_to_append, index=[0])
                
                # Use concat to append the data
                data = pd.concat([data, df_to_append], axis=0,ignore_index=True)
            res = requests.get(f"{url}/r/{subreddit}/new", headers=headers, params={"limit": 100, "after": res.json()['data']['after']})
        except:
            return data

In [120]:
df = get_reddit_data(subreddit="Investing",url=url, headers=headers)

In [123]:
# df.replace('|', '',inplace=True,regex=True)

In [125]:
# import os

# # Define the directory
# directory = '../data'

# # Check if the directory exists
# if not os.path.exists(directory):
#     # If the directory does not exist, create it
#     os.makedirs(directory)

#     df.to_csv(f'{directory}/reddit.csv',sep="|",index=False)

In [126]:
nlp = spacy.load('en_core_web_trf')

In [127]:
text = res.json()['data']['children'][8]['data']['selftext']

In [128]:
doc = nlp(text)

In [129]:
from spacy import displacy
displacy.render(doc,style='ent')

In [134]:
orgs = []
for ent in doc.ents:
    if ent.label_ == 'ORG':
        orgs.append(ent.text)
orgs

['FMC', 'FMC', 'FMC Corporation', 'FMC']

In [ ]:
pd.read_csv('../data/reddit.csv',sep='!')

In [139]:
def extract_entity(text):
    doc = nlp(text)
    orgs = []
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            orgs.append(ent.text)
    return orgs

In [ ]:
df['selftext'].apply(extract_entity)

Token indices sequence length is longer than the specified maximum sequence length for this model (1349 > 512). Running this sequence through the model will result in indexing errors
